# Optimasi Sistem Penerimaan credit di HomeCredit

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Membuka DataFrame utntuk data train
df = pd.read_csv("Downloads\home-credit-default-risk\\application_train.csv",
                 usecols = ["TARGET","NAME_FAMILY_STATUS","CNT_CHILDREN", "NAME_INCOME_TYPE", "AMT_INCOME_TOTAL","AMT_CREDIT", "AMT_ANNUITY", "NAME_HOUSING_TYPE", "ORGANIZATION_TYPE", "NAME_CONTRACT_TYPE", "AMT_REQ_CREDIT_BUREAU_YEAR"])

In [3]:
df.head()

,TARGET,NAME_CONTRACT_TYPE,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,ORGANIZATION_TYPE,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,0,202500.0,406597.5,24700.5,Working,Single / not married,House / apartment,Business Entity Type 3,1.0
1,0,Cash loans,0,270000.0,1293502.5,35698.5,State servant,Married,House / apartment,School,0.0
2,0,Revolving loans,0,67500.0,135000.0,6750.0,Working,Single / not married,House / apartment,Government,0.0
3,0,Cash loans,0,135000.0,312682.5,29686.5,Working,Civil marriage,House / apartment,Business Entity Type 3,NaN
4,0,Cash loans,0,121500.0,513000.0,21865.5,Working,Single / not married,House / apartment,Religion,0.0


In [4]:
df.shape

(307511, 11)

### Data Prepocessing

In [5]:
df.isna().sum()

TARGET                            0
NAME_CONTRACT_TYPE                0
CNT_CHILDREN                      0
AMT_INCOME_TOTAL                  0
AMT_CREDIT                        0
AMT_ANNUITY                      12
NAME_INCOME_TYPE                  0
NAME_FAMILY_STATUS                0
NAME_HOUSING_TYPE                 0
ORGANIZATION_TYPE                 0
AMT_REQ_CREDIT_BUREAU_YEAR    41519
dtype: int64

In [6]:
df["AMT_ANNUITY"] = df["AMT_ANNUITY"].fillna(0)
df["AMT_REQ_CREDIT_BUREAU_YEAR"] = df["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(0)

In [7]:
df.isna().sum()

TARGET                        0
NAME_CONTRACT_TYPE            0
CNT_CHILDREN                  0
AMT_INCOME_TOTAL              0
AMT_CREDIT                    0
AMT_ANNUITY                   0
NAME_INCOME_TYPE              0
NAME_FAMILY_STATUS            0
NAME_HOUSING_TYPE             0
ORGANIZATION_TYPE             0
AMT_REQ_CREDIT_BUREAU_YEAR    0
dtype: int64

In [8]:
dfs = df.drop(columns = ["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "TARGET", "CNT_CHILDREN", "AMT_REQ_CREDIT_BUREAU_YEAR"])
dfs.head()

,NAME_CONTRACT_TYPE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,ORGANIZATION_TYPE
0,Cash loans,Working,Single / not married,House / apartment,Business Entity Type 3
1,Cash loans,State servant,Married,House / apartment,School
2,Revolving loans,Working,Single / not married,House / apartment,Government
3,Cash loans,Working,Civil marriage,House / apartment,Business Entity Type 3
4,Cash loans,Working,Single / not married,House / apartment,Religion


In [9]:
dfs = dfs.apply(lambda x: pd.factorize(x)[0])

In [10]:
dfs.head()

,NAME_CONTRACT_TYPE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,ORGANIZATION_TYPE
0,0,0,0,0,0
1,0,1,1,0,1
2,1,0,0,0,2
3,0,0,2,0,0
4,0,0,0,0,3


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
dff = df[["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY"]]
scaler = StandardScaler()
scaler.fit(dff)
dffs = scaler.transform(dff)
dff = pd.DataFrame(dffs, index = dff.index, columns = dff.columns)

In [12]:
dff.head()

,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY
0,0.142129,-0.478095,-0.166065
1,0.426792,1.725450,0.592710
2,-0.427196,-1.152888,-1.404507
3,-0.142533,-0.711430,0.177929
4,-0.199466,-0.213734,-0.361658


In [13]:
dfi = df[["TARGET", "CNT_CHILDREN", "AMT_REQ_CREDIT_BUREAU_YEAR"]]
dfi.head()

,TARGET,CNT_CHILDREN,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,0,1.0
1,0,0,0.0
2,0,0,0.0
3,0,0,0.0
4,0,0,0.0


In [14]:
df = pd.concat([dfi,dfs,dff], axis = 1)
df.head()

,TARGET,CNT_CHILDREN,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,ORGANIZATION_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY
0,1,0,1.0,0,0,0,0,0,0.142129,-0.478095,-0.166065
1,0,0,0.0,0,1,1,0,1,0.426792,1.725450,0.592710
2,0,0,0.0,1,0,0,0,2,-0.427196,-1.152888,-1.404507
3,0,0,0.0,0,0,2,0,0,-0.142533,-0.711430,0.177929
4,0,0,0.0,0,0,0,0,3,-0.199466,-0.213734,-0.361658


In [15]:
x = df.drop(columns = ["TARGET"])
y = df["TARGET"]

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y , test_size = 0.2,random_state = 1)

## Data Processing

### Pemodelan data dengan algoritma naive bayes

In [17]:
from sklearn.naive_bayes import GaussianNB
model_Gaussian = GaussianNB()
model_Gaussian.fit(x_train, y_train)

GaussianNB()

In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score 

In [19]:
y_pred = model_Gaussian.predict(x_test)
cm =confusion_matrix(y_test,y_pred)
print("confusion matrix")
print(cm)
akurasi=classification_report(y_test,y_pred)
print("tingkat akurasi algoritma Naive Bayes")
print("Akurasi :", akurasi)
akurasi=accuracy_score(y_test,y_pred) 
print("Tingkat Akurasi :%d persen"%(akurasi*100))

confusion matrix
[[56112   421]
 [ 4943    27]]
tingkat akurasi algoritma Naive Bayes
Akurasi :               precision    recall  f1-score   support

           0       0.92      0.99      0.95     56533
           1       0.06      0.01      0.01      4970

    accuracy                           0.91     61503
   macro avg       0.49      0.50      0.48     61503
weighted avg       0.85      0.91      0.88     61503

Tingkat Akurasi :91 persen


In [20]:
from sklearn.model_selection import cross_val_score
scores1 = cross_val_score(model_Gaussian, x_test, y_test, cv = 10)
scores1.mean()

0.917353643336466

### Pemodelan data dengan algoritma Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression
model_Logistic = LogisticRegression()
model_Logistic.fit(x_train, y_train)

LogisticRegression()

In [22]:
y_pred2 = model_Logistic.predict(x_test)
cm2 =confusion_matrix(y_test,y_pred2)
print("confusion matrix")
print(cm)
akurasi2=classification_report(y_test,y_pred2)
print("tingkat akurasi algoritma LogisticRegression")
print("Akurasi :", akurasi2)
akurasi2=accuracy_score(y_test,y_pred2) 
print("Tingkat Akurasi :%d persen"%(akurasi2*100))

confusion matrix
[[56112   421]
 [ 4943    27]]
tingkat akurasi algoritma LogisticRegression
Akurasi :               precision    recall  f1-score   support

           0       0.92      1.00      0.96     56533
           1       0.00      0.00      0.00      4970

    accuracy                           0.92     61503
   macro avg       0.46      0.50      0.48     61503
weighted avg       0.84      0.92      0.88     61503

Tingkat Akurasi :91 persen


C:\Users\ACER\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
from sklearn.model_selection import cross_val_score
scores2 = cross_val_score(model_Logistic, x_test, y_test, cv = 10)
scores2.mean()

0.9191909333269891

## Memprediksi data baru

In [24]:
df_new = pd.read_csv("Downloads\home-credit-default-risk\\application_test.csv",
                 usecols = ["NAME_FAMILY_STATUS","CNT_CHILDREN", "NAME_INCOME_TYPE", "AMT_INCOME_TOTAL","AMT_CREDIT", "AMT_ANNUITY", "NAME_HOUSING_TYPE", "ORGANIZATION_TYPE", "NAME_CONTRACT_TYPE", "AMT_REQ_CREDIT_BUREAU_YEAR"])

In [25]:
df_new.shape

(48744, 10)

In [26]:
df_new.isna().sum()

NAME_CONTRACT_TYPE               0
CNT_CHILDREN                     0
AMT_INCOME_TOTAL                 0
AMT_CREDIT                       0
AMT_ANNUITY                     24
NAME_INCOME_TYPE                 0
NAME_FAMILY_STATUS               0
NAME_HOUSING_TYPE                0
ORGANIZATION_TYPE                0
AMT_REQ_CREDIT_BUREAU_YEAR    6049
dtype: int64

In [27]:
df_new["AMT_ANNUITY"] = df_new["AMT_ANNUITY"].fillna(0)
df_new["AMT_REQ_CREDIT_BUREAU_YEAR"] = df_new["AMT_REQ_CREDIT_BUREAU_YEAR"].fillna(0)

In [28]:
dfs_new = df_new.drop(columns = ["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "CNT_CHILDREN", "AMT_REQ_CREDIT_BUREAU_YEAR"])
dfs_new.head()

,NAME_CONTRACT_TYPE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,ORGANIZATION_TYPE
0,Cash loans,Working,Married,House / apartment,Kindergarten
1,Cash loans,Working,Married,House / apartment,Self-employed
2,Cash loans,Working,Married,House / apartment,Transport: type 3
3,Cash loans,Working,Married,House / apartment,Business Entity Type 3
4,Cash loans,Working,Married,House / apartment,Business Entity Type 3


In [29]:
dfs_new = dfs_new.apply(lambda x: pd.factorize(x)[0])

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
dff_new = df_new[["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY"]]
scaler = StandardScaler()
scaler.fit(dff_new)
dffs_new = scaler.transform(dff_new)
dff_new = pd.DataFrame(dffs_new, index = dff_new.index, columns = dff_new.columns)

In [31]:
dfi_new = df_new[["CNT_CHILDREN", "AMT_REQ_CREDIT_BUREAU_YEAR"]]

### Prediksi data baru dengan Naive Bayes

In [32]:
xnew = pd.concat([dfi_new,dfs_new,dff_new], axis = 1)
ynew = model_Gaussian.predict(xnew)
ynew

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [33]:
ynew = pd.DataFrame(ynew, index = y[:48744].index, columns = {'HASIL'})
ynew

,HASIL
0,0
1,0
2,0
3,0
4,0
...,...
48739,0
48740,0
48741,0
48742,0


In [34]:
data_ID = pd.read_csv("Downloads\home-credit-default-risk\\application_test.csv",
                 usecols = ["SK_ID_CURR"])

In [35]:
hasil_prediksi = pd.concat([data_ID, ynew], axis = 1)

In [36]:
hasil_prediksi[:11]

,SK_ID_CURR,HASIL
0,100001,0
1,100005,0
2,100013,0
3,100028,0
4,100038,0
5,100042,0
6,100057,0
7,100065,0
8,100066,1
9,100067,0


### Prediksi data baru dengan logistic regression

In [37]:
ynew2 = model_Logistic.predict(xnew)
ynew2

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [38]:
ynew2 = pd.DataFrame(ynew2, index = y[:48744].index, columns = {'HASIL'})
ynew2.head()

,HASIL
0,0
1,0
2,0
3,0
4,0


In [39]:
hasil_prediksi2 = pd.concat([data_ID, ynew2], axis = 1)
hasil_prediksi[:11]

,SK_ID_CURR,HASIL
0,100001,0
1,100005,0
2,100013,0
3,100028,0
4,100038,0
5,100042,0
6,100057,0
7,100065,0
8,100066,1
9,100067,0
